# Notebook 2: Datenanalyse

**Projekt:** Analyse der Haushaltsausgaben in der Schweiz (2006-2022)  
**Autor:** CAS Information Engineering - ZHAW  
**Version:** 1.0  
**Datum:** 14. Oktober 2025

---

## Ziel dieses Notebooks

Dieses Notebook implementiert die **Datenanalyse** gemäß den funktionalen Anforderungen FA-02.1 bis FA-02.7:

- **FA-02.1:** Laden der bereinigten Daten
- **FA-02.2:** Analyse der Gesamtausgabenentwicklung
- **FA-02.3:** Analyse nach Altersgruppen
- **FA-02.4:** Analyse nach Haushaltstyp
- **FA-02.5:** Top-Wachstumskategorien identifizieren
- **FA-02.6:** Statistische Kennzahlen berechnen
- **FA-02.7:** Export der Analyseergebnisse

---

## Inhaltsverzeichnis

1. [Initialisierung und Laden der bereinigten Daten](#1-initialisierung)
2. [Gesamtausgabenentwicklung](#2-gesamtausgaben)
3. [Analyse nach Altersgruppen](#3-altersgruppen)
4. [Analyse nach Haushaltstyp](#4-haushaltstyp)
5. [Top-Wachstumskategorien](#5-wachstum)
6. [Statistische Kennzahlen](#6-statistik)
7. [Export der Analyseergebnisse](#7-export)
8. [Zusammenfassung](#8-zusammenfassung)

---

## 1. Initialisierung und Laden der bereinigten Daten <a id='1-initialisierung'></a>

### 1.1 Import der Bibliotheken

Wir importieren die notwendigen Bibliotheken für die Datenanalyse.

In [ ]:
import pandas as pd
import numpy as np
import os
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Versionen anzeigen für Reproduzierbarkeit
print(f"Pandas Version: {pd.__version__}")
print(f"NumPy Version: {np.__version__}")

### 1.2 Pfade definieren

In [ ]:
# Basis-Pfade definieren
BASE_DIR = Path.cwd().parent
PROCESSED_DATA_DIR = BASE_DIR / 'data' / 'processed'
OUTPUT_TABLES_DIR = BASE_DIR / 'outputs' / 'tables'

# Output-Ordner erstellen, falls nicht vorhanden
OUTPUT_TABLES_DIR.mkdir(parents=True, exist_ok=True)

print(f"Processed Data Directory: {PROCESSED_DATA_DIR}")
print(f"Output Tables Directory: {OUTPUT_TABLES_DIR}")
print(f"Processed-Ordner existiert: {PROCESSED_DATA_DIR.exists()}")
print(f"Output-Ordner existiert: {OUTPUT_TABLES_DIR.exists()}")

### 1.3 Laden der bereinigten CSV-Dateien (FA-02.1)

In [ ]:
def load_cleaned_data(filename, data_dir=PROCESSED_DATA_DIR):
    try:
        file_path = data_dir / filename
        df = pd.read_csv(file_path)
        print(f"Erfolgreich geladen: {filename}")
        print(f"  Shape: {df.shape}")
        return df
    except FileNotFoundError:
        print(f"FEHLER: Datei nicht gefunden: {filename}")
        print(f"  Bitte führen Sie zuerst Notebook 01 aus!")
        return None
    except Exception as e:
        print(f"FEHLER beim Laden von {filename}: {e}")
        return None

In [ ]:
print("LADEN DER BEREINIGTEN DATEN")
print("="*60)

df_gesamt = load_cleaned_data('clean_gesamtausgaben.csv')
df_alter = load_cleaned_data('clean_alter.csv')
df_haushaltstyp = load_cleaned_data('clean_haushaltstyp.csv')

print("="*60)
print("Laden abgeschlossen!")

### 1.4 Überprüfung der geladenen Daten

In [ ]:
if df_gesamt is not None:
    print("GESAMTAUSGABEN - Überprüfung")
    df_gesamt.info()
    display(df_gesamt.head())

In [ ]:
if df_alter is not None:
    print("AUSGABEN NACH ALTER - Überprüfung")
    df_alter.info()
    display(df_alter.head())
    if 'altersgruppe' in df_alter.columns:
        print("Altersgruppen:", df_alter['altersgruppe'].unique())

In [ ]:
if df_haushaltstyp is not None:
    print("AUSGABEN NACH HAUSHALTSTYP - Überprüfung")
    df_haushaltstyp.info()
    display(df_haushaltstyp.head())
    if 'haushaltstyp' in df_haushaltstyp.columns:
        print("Haushaltstypen:", df_haushaltstyp['haushaltstyp'].unique())

---

## 2. Gesamtausgabenentwicklung (FA-02.2) <a id='2-gesamtausgaben'></a>

In [ ]:
if df_gesamt is not None and 'periode' in df_gesamt.columns and 'betrag_chf' in df_gesamt.columns:
    ausgaben_pro_jahr = df_gesamt.groupby('periode')['betrag_chf'].sum().reset_index()
    ausgaben_pro_jahr.columns = ['jahr', 'gesamtausgaben_chf']
    ausgaben_pro_jahr['veraenderung_prozent'] = ausgaben_pro_jahr['gesamtausgaben_chf'].pct_change() * 100
    ausgaben_pro_jahr['veraenderung_absolut'] = ausgaben_pro_jahr['gesamtausgaben_chf'].diff()
    
    print("GESAMTAUSGABENENTWICKLUNG 2006-2022")
    print("="*80)
    display(ausgaben_pro_jahr)
    
    if len(ausgaben_pro_jahr) > 1:
        erste_jahr = ausgaben_pro_jahr.iloc[0]
        letzte_jahr = ausgaben_pro_jahr.iloc[-1]
        gesamtwachstum = ((letzte_jahr['gesamtausgaben_chf'] - erste_jahr['gesamtausgaben_chf']) / erste_jahr['gesamtausgaben_chf'] * 100)
        
        print(f"Ausgaben {int(erste_jahr['jahr'])}: CHF {erste_jahr['gesamtausgaben_chf']:,.2f}")
        print(f"Ausgaben {int(letzte_jahr['jahr'])}: CHF {letzte_jahr['gesamtausgaben_chf']:,.2f}")
        print(f"Gesamtwachstum: {gesamtwachstum:.2f}%")
        print(f"Durchschnittliches Wachstum: {ausgaben_pro_jahr['veraenderung_prozent'].mean():.2f}%")
else:
    print("Daten für Gesamtausgabenentwicklung nicht verfügbar")

---

## 3. Analyse nach Altersgruppen (FA-02.3) <a id='3-altersgruppen'></a>

In [ ]:
if df_alter is not None and 'altersgruppe' in df_alter.columns and 'betrag_chf' in df_alter.columns:
    ausgaben_nach_alter = df_alter.groupby('altersgruppe')['betrag_chf'].agg([('durchschnitt_chf', 'mean'), ('summe_chf', 'sum'), ('anzahl_eintraege', 'count')]).reset_index()
    ausgaben_nach_alter = ausgaben_nach_alter.sort_values('durchschnitt_chf', ascending=False)
    
    print("AUSGABEN NACH ALTERSGRUPPEN")
    print("="*80)
    display(ausgaben_nach_alter)
    
    hoechste_gruppe = ausgaben_nach_alter.iloc[0]
    niedrigste_gruppe = ausgaben_nach_alter.iloc[-1]
    
    print(f"Altersgruppe mit HÖCHSTEN Ausgaben: {hoechste_gruppe['altersgruppe']}")
    print(f"  Durchschnitt: CHF {hoechste_gruppe['durchschnitt_chf']:,.2f}")
    print(f"Altersgruppe mit NIEDRIGSTEN Ausgaben: {niedrigste_gruppe['altersgruppe']}")
    print(f"  Durchschnitt: CHF {niedrigste_gruppe['durchschnitt_chf']:,.2f}")
else:
    print("Daten für Altersgruppen-Analyse nicht verfügbar")

---

## 4. Analyse nach Haushaltstyp (FA-02.4) <a id='4-haushaltstyp'></a>

In [ ]:
if df_haushaltstyp is not None and 'haushaltstyp' in df_haushaltstyp.columns and 'betrag_chf' in df_haushaltstyp.columns:
    ausgaben_nach_haushalt = df_haushaltstyp.groupby('haushaltstyp')['betrag_chf'].agg([('durchschnitt_chf', 'mean'), ('summe_chf', 'sum'), ('anzahl_eintraege', 'count')]).reset_index()
    ausgaben_nach_haushalt = ausgaben_nach_haushalt.sort_values('durchschnitt_chf', ascending=False)
    
    print("AUSGABEN NACH HAUSHALTSTYP")
    print("="*80)
    display(ausgaben_nach_haushalt)
else:
    print("Daten für Haushaltstyp-Analyse nicht verfügbar")

In [ ]:
if df_haushaltstyp is not None and 'haushaltstyp' in df_haushaltstyp.columns and 'kategorie' in df_haushaltstyp.columns:
    haushaltstypen_unique = df_haushaltstyp['haushaltstyp'].unique()
    print("Verfügbare Haushaltstypen:", haushaltstypen_unique)
    
    familie_typen = [ht for ht in haushaltstypen_unique if 'familie' in str(ht).lower() or 'paar mit kind' in str(ht).lower()]
    single_typen = [ht for ht in haushaltstypen_unique if 'single' in str(ht).lower() or 'einpersonenhaushalt' in str(ht).lower() or '1 person' in str(ht).lower()]
    
    if familie_typen:
        df_familie = df_haushaltstyp[df_haushaltstyp['haushaltstyp'].isin(familie_typen)]
        top_kategorien_familie = df_familie.groupby('kategorie')['betrag_chf'].mean().sort_values(ascending=False).head(3)
        print("TOP 3 AUSGABENKATEGORIEN - FAMILIE")
        for idx, (kategorie, betrag) in enumerate(top_kategorien_familie.items(), 1):
            print(f"{idx}. {kategorie}: CHF {betrag:,.2f}")
    
    if single_typen:
        df_single = df_haushaltstyp[df_haushaltstyp['haushaltstyp'].isin(single_typen)]
        top_kategorien_single = df_single.groupby('kategorie')['betrag_chf'].mean().sort_values(ascending=False).head(3)
        print("TOP 3 AUSGABENKATEGORIEN - SINGLE")
        for idx, (kategorie, betrag) in enumerate(top_kategorien_single.items(), 1):
            print(f"{idx}. {kategorie}: CHF {betrag:,.2f}")

---

## 5. Top-Wachstumskategorien (FA-02.5) <a id='5-wachstum'></a>

In [ ]:
if df_gesamt is not None and 'periode' in df_gesamt.columns and 'kategorie' in df_gesamt.columns:
    df_2006 = df_gesamt[df_gesamt['periode'] == 2006].groupby('kategorie')['betrag_chf'].mean()
    df_2022 = df_gesamt[df_gesamt['periode'] == 2022].groupby('kategorie')['betrag_chf'].mean()
    
    wachstum_kategorien = pd.DataFrame({'mittelwert_2006': df_2006, 'mittelwert_2022': df_2022})
    wachstum_kategorien['veraenderung_prozent'] = ((wachstum_kategorien['mittelwert_2022'] - wachstum_kategorien['mittelwert_2006']) / wachstum_kategorien['mittelwert_2006'] * 100)
    wachstum_kategorien = wachstum_kategorien.sort_values('veraenderung_prozent', ascending=False)
    
    print("TOP 5 WACHSTUMSKATEGORIEN 2006-2022")
    print("="*80)
    top_5_wachstum = wachstum_kategorien.head(5)
    display(top_5_wachstum)
    
    for idx, (kategorie, row) in enumerate(top_5_wachstum.iterrows(), 1):
        print(f"{idx}. {kategorie}: {row['veraenderung_prozent']:.2f}% Wachstum")
else:
    print("Daten für Wachstumsanalyse nicht verfügbar")

---

## 6. Statistische Kennzahlen (FA-02.6) <a id='6-statistik'></a>

In [ ]:
if df_gesamt is not None and 'kategorie' in df_gesamt.columns:
    interessante_kategorie = 'Wohnen'
    
    if interessante_kategorie in df_gesamt['kategorie'].values:
        df_wohnen = df_gesamt[df_gesamt['kategorie'] == interessante_kategorie]
        
        print(f"STATISTISCHE KENNZAHLEN - Kategorie: {interessante_kategorie}")
        print("="*80)
        display(df_wohnen['betrag_chf'].describe())
    else:
        print(f"Kategorie '{interessante_kategorie}' nicht in Daten gefunden")
        print("Verfügbare Kategorien:", df_gesamt['kategorie'].unique()[:10])
else:
    print("Daten für statistische Analyse nicht verfügbar")

---

## 7. Export der Analyseergebnisse (FA-02.7) <a id='7-export'></a>

In [ ]:
if 'wachstum_kategorien' in locals():
    wachstum_kategorien_export = wachstum_kategorien.reset_index()
    wachstum_kategorien_export.columns = ['kategorie', 'mittelwert_2006', 'mittelwert_2022', 'veraenderung_prozent']
    
    output_file = OUTPUT_TABLES_DIR / 'zusammenfassung.csv'
    wachstum_kategorien_export.to_csv(output_file, index=False, encoding='utf-8')
    
    print("EXPORT ERFOLGREICH")
    print("="*80)
    print(f"Datei gespeichert: {output_file}")
    print(f"Anzahl Zeilen: {len(wachstum_kategorien_export)}")
    
    print("Erste Zeilen der exportierten Daten:")
    display(wachstum_kategorien_export.head(10))
else:
    print("Keine Wachstumsdaten zum Export verfügbar")

---

## 8. Zusammenfassung <a id='8-zusammenfassung'></a>

### Durchgeführte Schritte

In diesem Notebook wurden folgende Aufgaben erfolgreich durchgeführt:

1. **FA-02.1:** Bereinigte Daten aus CSV-Dateien geladen
2. **FA-02.2:** Gesamtausgabenentwicklung 2006-2022 analysiert
3. **FA-02.3:** Ausgaben nach Altersgruppen analysiert
4. **FA-02.4:** Ausgaben nach Haushaltstyp verglichen
5. **FA-02.5:** Top-Wachstumskategorien identifiziert
6. **FA-02.6:** Statistische Kennzahlen berechnet
7. **FA-02.7:** Analyseergebnisse als CSV exportiert

### Ausgabedateien

Die folgende CSV-Datei wurde im Ordner `outputs/tables/` erstellt:

- `zusammenfassung.csv` - Wachstumsanalyse 2006-2022

### Nächste Schritte

Die Analyseergebnisse sind nun bereit für:

- **Notebook 3:** Visualisierung (FA-03)
- **Notebook 4:** Business Case Dokumentation (FA-04)